## Dash Tutorial

Dash is a web application framework for Python that allows you to create interactive web applications. It's built on top of Plotly, so you can use all of the Plotly functionality you've learned here for Dash app development. Dash plots tends to allow for far more complex interactivity than Plotly-only plots, as you can more easily combine multiple features like sliders and buttons (discussed a little later on) to work in tandem due to its dynamic "callback" capabilities. You can even have plots that dynamically adjust their content based on point(s) you've selected in another plot.

There's a phenomenal YouTube tutorial on Dash by a Plotly/Dash community manager that I recommend you watch  before you try and go through this example. Primarily focus on the first 3 videos, as the latter 3 are pretty technical and specific about more advanced features: https://www.youtube.com/playlist?list=PLYD54mj9I2JevdabetHsJ3RLCeMyBNKYV


Here are some links to the Dash documentation as well:

Here's a link to the Dash documentation: https://dash.plotly.com/

Basic ~20 minute Dash tutorial: https://dash.plotly.com/tutorial

Useful interactivity features: https://dash.plotly.com/dash-core-components

Dash App examples: https://dash-example-index.herokuapp.com/





## Imports

In [1]:
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default='notebook'

## Dash Application Example

The following code is an example of a Dash app with just about every single interactive bell and whistle you may want to include in your app. It's a bit of a mess, but it's a good starting point for you to see how you can include multiple different features into your app. The data for the plot includes multiple different country-level data values (like GDP, fertility, imports, etc.) 

Disclaimer, I did not make the majority of it (the original is the second example of https://dash.plotly.com/interactive-graphing), and just modified the original code slightly to include a printout of the data table at the top and a range slider for the year for the "x/y vs value" secondary plots.


In [2]:
# Load data
df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')
df.head()

,Country Name,Indicator Name,Year,Value
0,Arab World,"Agriculture, value added (% of GDP)",1962,NaN
1,Arab World,CO2 emissions (metric tons per capita),1962,0.760996
2,Arab World,Domestic credit provided by financial sector (...,1962,18.168690
3,Arab World,Electric power consumption (kWh per capita),1962,NaN
4,Arab World,Energy use (kg of oil equivalent per capita),1962,NaN


In [4]:
# You can load and use a pre-made styling sheet if you want
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Create the Dash app by passing __name__ and an external stylesheet if you have one you want to use
app = Dash(__name__, external_stylesheets=external_stylesheets)

# Start by creating the layout of the app, which typically involves creating an initial div (a div is a container for elements) 
# with the html.Div() method for all the elements you want to display in the app
app.layout = html.Div([
    # Create a div to hold the data table preview
    html.Div([
        dash_table.DataTable( #Create a table to display the data
            data=df.to_dict('records'), #Pass the data to the table and convert it to a dictionary 
            columns=[{'name': i, 'id': i} for i in df.columns], #Create the columns for the table
            page_size=6, #Set the maximum number of displayed rows to 6
            filter_action="native", #Allows you to filter each column, though this is NOT implmented in a robust fashion, its jsut for demonstration
            style_table={'overflowX': 'auto'}, 
            style_cell={'textAlign': 'left'} 
            ),
    ]),
    
    # Create a div to hold the dropdowns and radio buttons for the x-axis and y-axis
    html.Div([ 
        #Create a div to hold the dropdown and radio buttons specifically for the x-axis
        html.Div([
            dcc.Dropdown( #Create a dropdown for the x-axis containing the multiple different unique values of the 'Indicator Name' column
                options=df['Indicator Name'].unique(), #Set the dropdown's selectable options to the unique values of the 'Indicator Name' column
                value='Fertility rate, total (births per woman)', #Set the default value of the dropdown to 'Fertility rate'
                id='crossfilter-xaxis-column', #Set the id of the dropdown to 'crossfilter-xaxis-column' for referencing in the callbacks
            ),
            dcc.RadioItems( #Create radio buttons for the x-axis scale
                options=['Linear', 'Log'], #Set the options of the radio buttons to 'Linear' and 'Log'
                value='Linear', #Set the default value of the radio buttons to 'Linear'
                id='crossfilter-xaxis-type', #Set the id of the radio buttons to 'crossfilter-xaxis-type' for referencing in the callbacks
                labelStyle={'display': 'inline-block', 'marginTop': '5px'} #Style the labels of the radio buttons
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}), #Style the div containing the dropdowns and radio buttons for the x-axis

        #Same as the above div, but now for the y-axis dropdown and radio buttons
        html.Div([ 
            dcc.Dropdown(
                options=df['Indicator Name'].unique(),
                value='Life expectancy at birth, total (years)',
                id='crossfilter-yaxis-column'
            ),
            dcc.RadioItems(
                options=['Linear', 'Log'],
                value='Linear',
                id='crossfilter-yaxis-type',
                labelStyle={'display': 'inline-block', 'marginTop': '5px'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
        
    ], style={'padding': '10px 5px'}), #Style the div containing the dropdowns and radio buttons for both axes

    #Create a div to hold the primary scatter plot of the app
    html.Div([
        dcc.Graph( #Create a scatter plot for the y vs x axis
            id='crossfilter-indicator-scatter', #Set the id of the scatter plot to 'crossfilter-indicator-scatter' for referencing in the callbacks
            hoverData={'points': [{'customdata': 'Japan'}]} #Set the default hover data to the data point for 'Japan'
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}), #Style the div containing the scatter plot
    
    #Create a div to hold the time series plots for the x-axis and y-axis
    html.Div([
        dcc.Graph(id='x-time-series'), #Create a time series plot for the x-axis
        dcc.Graph(id='y-time-series'), #Create a time series plot for the y-axis
    ], style={'display': 'inline-block', 'width': '49%'}), #Style the div containing the time series plots

    #Create a div to hold both the data slider controlling the year you want the data from to be displayed, 
    #and the range slider for range of years to view in the x and y time series plots
    html.Div([
        #Create a div to hold the data slider for the year
        html.Div([
            html.Label('Year:', style={'color': 'red'}), #Create a label for the year slider
            dcc.Slider( #Create a slider for the year
                df['Year'].min(), #Set the minimum value of the slider to the minimum year in the data
                df['Year'].max(), #Set the maximum value of the slider to the maximum year in the data
                step=None, #This sets the steo size between ticks on the slider to None, meaning the only ticks that will be displayed are the years in the data. 
                            #If you set a numeric step size, it's possible that ticks will be created for values that don't exist in the data
                id='crossfilter-year--slider', #Set the id of the slider to 'crossfilter-year--slider' for referencing in the callbacks
                value=df['Year'].max(), #Set the default value of the slider to the maximum year in the data
                marks={str(year): str(year) for year in df['Year'].unique()}, #Set the mark labels of the slider to the unique years in the data
                tooltip={"placement": "bottom", "always_visible": True} #This adds a tooltip to the bottom of the slider selector that displays the current value of the slider
            )
        ], style={'width': '48%', 'display': 'inline-block', 'padding': '0px 10px 20px 10px'}), #Style the div containing the year slider
        
        #Create a div to hold the range slider for the x and y time series plots
        html.Div([
            html.Label('Year Range Slider:', style={'color': 'green'}), #Create a label for the range slider
            dcc.RangeSlider( #Create a range slider for the year range
                df['Year'].min(), #Set the minimum value of the range slider to the minimum year in the data
                df['Year'].max(), #Set the maximum value of the range slider to the maximum year in the data
                step=1, #Set the step size between ticks on the range slider to 1, so that ever tick corresponds to a year
                id='crossfilter-year-range--slider', #Set the id of the range slider to 'crossfilter-year-range--slider' for referencing in the callbacks
                value=[df['Year'].min(), df['Year'].max()], #Set the default value of the range slider to the minimum and maximum years in the data
                marks={str(year): str(year) for year in df['Year'].unique()}, #Set the mark labels of the range slider to the unique years in the data
                tooltip={"placement": "bottom", "always_visible": True} #This adds a tooltip to the bottom of the left and right slider selectors that displays the current value of the selector
            )
        ], style={'width': '48%', 'display': 'inline-block', 'padding': '0px 10px 20px 10px'}) #Style the div containing the range slider
    ])
])

# Create a function to generate a new time series plot for use in the callbacks that update the x and y time series plots.
# It's being passed the new data frame from the callback, the axis type from the radio button selection, and the title of 
# the plot taken from the country name of the hover data. Hopefully the Plotly code looks familiar to you by now.
def create_time_series(dff, axis_type, title):
    fig = px.scatter(dff, x='Year', y='Value')
    fig.update_traces(mode='lines+markers')
    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(type='linear' if axis_type == 'Linear' else 'log') 
    fig.add_annotation(x=0, y=0.85, xanchor='left', yanchor='bottom',
                       xref='paper', yref='paper', showarrow=False, align='left',
                       text=title)
    fig.update_layout(height=225, margin={'l': 20, 'b': 30, 'r': 10, 't': 10})
    return fig

# Create the callback functions that will update the main y vs x scatter plot. Since it's alterable by all of them, 
# it needs the currently selected x radio button, y radio button, x dropdown, y dropdown, and year slider values as inputs.
# It will output the updated scatter plot figure. Callback functions are triggered whenever Dash detects a change in 
# any of these input values, and will update the output values accordingly. The callback function is defined by the "@callback"
# decorator, as well as a corrsponding decorator function that you build that takes the updated input(s) and returns the updated output(s).
# This decorator function requires that ever single callback input be passed as an argument in the same order as they are listed in the callback decorator. 
@callback(
    Output(component_id='crossfilter-indicator-scatter', component_property='figure'), # The main scatter plot
    Input(component_id='crossfilter-xaxis-column', component_property='value'), # The x-axis dropdown
    Input(component_id='crossfilter-yaxis-column', component_property='value'), # The y-axis dropdown
    Input(component_id='crossfilter-xaxis-type', component_property='value'), # The x-axis radio button
    Input(component_id='crossfilter-yaxis-type', component_property='value'), # The y-axis radio button
    Input(component_id='crossfilter-year--slider', component_property='value')) # The year slider
def update_graph(xaxis_column_name, #crossfilter-xaxis-column
                 yaxis_column_name, #crossfilter-yaxis-column
                 xaxis_type, #crossfilter-xaxis-type
                 yaxis_type, #crossfilter-yaxis-type
                 year_value #crossfilter-year--slider
                 ):
    dff = df[df['Year'] == year_value] #Filter the data frame to only include the year selected by the year slider
    ##Create a scatter plot with
    fig = px.scatter(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'], #the selected x-axis dropdown as the value of the x-axis column
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'], # the selected y-axis dropdown as the value of the y-axis column
            hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'] # the name of the currently selected country to be added into the hover window as the title
            )
    fig.update_traces(customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name']) # Update the scatter points to contain information about the country name for that point
    fig.update_xaxes(title=xaxis_column_name, type='linear' if xaxis_type == 'Linear' else 'log') # Set the x-axis scale based on the selected x-axis radio button value
    fig.update_yaxes(title=yaxis_column_name, type='linear' if yaxis_type == 'Linear' else 'log') # Set the y-axis scale based on the selected y-axis radio button value
    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest') # Update the layout of the scatter plot. The margin is the space around the plot, and the hovermode relates to the way the hover window is displayed
    return fig #Return the updated scatter plot figure

# Create the callback function that will update SPECIFICALLY the top time series vs x-axis plot. Since it takes the hover data of the main scatter plot points
# as an in put, it gets triggered whenever you mouse over a point on the main scatter plot. It also takes the x-axis dropdown values, x-axis radio button value,
# and the year range slider values as inputs. It outputs the updated x time series plot figure.
@callback(
    Output(component_id='x-time-series', component_property='figure'), # The x time series plot
    Input(component_id='crossfilter-indicator-scatter', component_property='hoverData'), # The hover data of the main scatter plot
    Input(component_id='crossfilter-xaxis-column', component_property='value'), # The x-axis dropdown
    Input(component_id='crossfilter-xaxis-type', component_property='value'), # The x-axis radio button
    Input(component_id='crossfilter-year-range--slider', component_property='value')) # The year range slider
def update_x_timeseries(hoverData, #crossfilter-indicator-scatter
                        xaxis_column_name, #crossfilter-xaxis-column
                        axis_type, #crossfilter-xaxis-type
                        year_range #crossfilter-year-range--slider
                        ):
    country_name = hoverData['points'][0]['customdata'] #Get the country name of the point you're hovering over. The hoverData object is a dictionary, and you can access the country name by indexing into the 'points' key, and then the 'customdata' key
    dff = df[(df['Country Name'] == country_name) & (df['Indicator Name'] == xaxis_column_name)] #Filter the data frame to only include the selected country and x-axis dropdown value
    dff = dff[(dff['Year'] >= year_range[0]) & (dff['Year'] <= year_range[1])] #Filter the data frame to only include the years specified by the year range slider
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name) #Create a title for the time series plot that includes the country name and x-axis dropdown value
    return create_time_series(dff, axis_type, title) #Return the updated time series plot figure, which is created by passing the relevant info to the create_time_series() function defined earlier

# The exact same as the above callback function, but now for the y time series plot, and taking the y-axis dropdown and radio button values as inputs instead
@callback(
    Output(component_id='y-time-series', component_property='figure'),
    Input(component_id='crossfilter-indicator-scatter', component_property='hoverData'),
    Input(component_id='crossfilter-yaxis-column', component_property='value'),
    Input(component_id='crossfilter-yaxis-type', component_property='value'),
    Input(component_id='crossfilter-year-range--slider', component_property='value'))
def update_y_timeseries(hoverData, #crossfilter-indicator-scatter
                        yaxis_column_name, #crossfilter-yaxis-column
                        axis_type, #crossfilter-yaxis-type
                        year_range #crossfilter-year-range--slider
                        ):
    country_name = hoverData['points'][0]['customdata'] 
    dff = df[(df['Country Name'] == country_name) & (df['Indicator Name'] == yaxis_column_name)]
    dff = dff[(dff['Year'] >= year_range[0]) & (dff['Year'] <= year_range[1])]
    return create_time_series(dff, axis_type, yaxis_column_name)

# Run the app
if __name__ == '__main__': #This is a standard Python construct that allows you to run the app only if the script is being run directly, and not imported as a module 
    port = 7062 #Set the port number for the app. This value doesn't matter, but it can be useful to change if you want to have multiple different dash apps running at the same time
    print(f"Dash app running on http://127.0.0.1:{port}/") #Print the URL of the app to the console so you can view the app in a browser if you prefer
    app.run(debug = True, port = port) #Run the app with debugging enabled and the specified port number


Dash app running on http://127.0.0.1:7062/
